In [1]:
import pandas as pd
import os
from sqlalchemy import create_engine
import time
import logging

# Set up logging
logging.basicConfig(
    filename="logs/ingestion_db.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)

# Create SQLite engine
engine = create_engine('sqlite:///inventory.db')

def ingest_db(df, table_name, engine):
    '''Ingest a DataFrame into a database table'''
    df.to_sql(table_name, con=engine, if_exists='replace', index=False,chunksize=1000)
    logging.info(f'Table {table_name} ingested successfully.')

def load_row_data():
    '''Load all CSVs from /data and insert them into the DB'''
    start = time.time()
    
    for file in os.listdir('data'):
        if file.endswith(".csv"):
            df = pd.read_csv(os.path.join('data', file))
            logging.info(f'Loading {file} into DataFrame')
            ingest_db(df, file[:-4], engine)
    
    end = time.time()
    elapsed = (end - start) / 60
    logging.info("--------------Ingestion Complete--------------")
    logging.info(f"Total Time taken: {elapsed:.2f} minutes")

if __name__ == '__main__':
    load_row_data()